# Part 4, Topic 2: CPA on Firmware Implementation of AES

**SUMMARY**: *Now that you've seen a CPA attack work, let's explore it in more detail. The goal of this lab will to do a CPA attack without using Analyzer.*


**LEARNING OUTCOMES:**
* Developing an algorithm based on a mathematical description
* Verify that correlation can be used to break a single byte of AES
* Extend the single byte attack to the rest of the key

**Requirements:** 
We'll be using a location value from the previous lab in this one, so make sure you've got it written down/recorded somewhere!

## Capturing Traces

Last time, we didn't go into much detail on capturing traces. This time, we'll take a closer look at the firmware. It uses something called simpleserial for communication, which is a simple communication protocol that we use for most of our firmware. We can use it to send different commands to the target and also to receive data. `simpleserial-aes` has two commands we care about:

* `'k'` - Set the key used for the AES implementation
* `'p'` - Send a plaintext to the target for it to encrypt. When the encryption is finished, it will respond with an `'r'` command.

We can use `target.simpleserial_write(<cmd>, <data>)` and `target.simpleserial.read(<cmd>, <dlen>)` to send and receive data. Basically, we want to:

1. Set the AES key used on the target using the `'k'` command
1. Arm the scope (`scope.arm()`)
1. Send the plaintext 
1. Capture the trace (`scope.capture()`). Note that this doesn't return the trace that gets captured, it returns whether (1) or not (0) the capture timed out
1. Read the ciphertext back

You can get the trace you just captured with `scope.get_last_trace()`. You'll want to do steps 2 through 5 multiple times.

## \#HARDWARE

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_2_1'

In [ ]:
# Connect to ChipWhisperer
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
# compile firmware
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

Unlike last time, where we used ChipWhisperer Projects, this time we'll be working with our traces as simple lists.

In [ ]:
trace_array = []
textin_array = []

This time, it'll be up to you to do most of the work capturing the traces. Don't forget to append your traces to `trace_array` and `text` to `textin_array`. If you get really stuck, you can check the [answer key](Answers.ipynb)

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
key, text = ktp.next()


# ###################
# Add your code here to set the key (Code Block 1)
# ###################
# Code Block 1
target.simpleserial_write('k', key)


for i in trange(50, desc='Capturing traces'):
    target.flush()
    key, text = ktp.next()
    # ###################
    # Add your code here to send the plaintext and capture the trace (Code Block 2)
    # ###################

You may want to plot some traces to make sure everything looks as expected:

In [ ]:
cw.plot(trace_array[0]) * cw.plot(trace_array[1])

In [ ]:
scope.dis()
target.dis()

## \#SIMULATED

If you don't have hardware, you can instead load previously captured traces instead:

In [ ]:
"""
import numpy as np
from tqdm.notebook import trange
import chipwhisperer as cw

aes_traces_50_tracedata = np.load(r"traces/lab4_2_traces.npy")
aes_traces_50_textindata = np.load(r"traces/lab4_2_textin.npy")
key = np.load(r"traces/lab4_2_key.npy")

trace_array = aes_traces_50_tracedata
textin_array = aes_traces_50_textindata
"""

## Attack Theory

We've seen in the slides that:

* The power consumed by an electronic device is related to the data being manipulated. 
* Storing a `1` consumes power, while storing a `0` doesn't. 
* This power consumption is linear. Storing 4 `1`s takes roughly twice as much power as storing 2 `1`s.
* In microcontrollers, this power consumption is generally unrelated to what was previously stored.

In a perfect scenario (no noise), if we measure the power consumption of the target when it's manipulating data, we end up knowing how many `1`s are being stored, known as the Hamming weight. Just the device loading the key doesn't do us much good here, as it's effectively impossible to know when the key is being loaded.

Instead, if the key is combined with some changing value that we know, we can use multiple power traces to fully figure out the key! We can do that by effectively replicating the function that generates our power trace, then running through with various key guesses until we get one that matches.

Let's try an example with 8-bit addition:

`power_trace = hamming_weight((key + trace) & 0xFF)`

In [ ]:
import random
secret_key = random.randint(0, 255)

In [ ]:
num_traces = 1

def hamming_weight(val):
    return bin(val).count("1")

def power_trace(text):
    key = secret_key # don't tell
    return hamming_weight((text + key) & 0xFF)

texts = [random.randint(0,255) for _ in range(num_traces)]
traces = [power_trace(text) for text in texts]
print(traces)

Next, let's take a look at figuring out the key. Basically, what we want to do is recreate the `power_trace()` operation, but replace the secret key with a guess. This looks like:

In [ ]:
import numpy as np

def trace_guesser(guess, text):
    return hamming_weight((text + guess) & 0xFF)

for guess in range(255):
    guess_traces = [trace_guesser(guess, text) for text in texts]
    if guess_traces == traces:
        print("Possible guess = {}".format(guess))

You should see that a single trace narrows down the key a bit. Now try increasing `num_traces` until you're 100% sure what the key is. As a bonus, try changing the operation in `power_trace()` and `trace_guesser()` to an XOR instead by replacing the `+` with `^` and running through a few times. You should see that the `+` operation was very reliable, but the `^` operation is less consistant.

This seems great and all, but our traces obviously won't be this perfect - there's going to be a bunch of noise from unrelated things happening and we won't know how much of the power traces is from the data we care about. We also won't know when in time the trace is happening. We need one final trick to counteract these issues, which is to use the linear relationship between the power trace and the data. Let's introduce a new, more realistic `power_trace` function that scales the effect of the Hamming weight, as well as adds a random amount of noise and generate some power traces:

In [ ]:
import chipwhisperer as cw
num_traces = 500 # larger number required here
scale_factor = 0.1 # how much the operation contributes to the power trace
noise_range = [-0.1, 0.1] # how much random noise to add

def power_trace(text):
    key = secret_key # don't tell
    data_component = hamming_weight((text + key) & 0xFF) * scale_factor
    noise_component = random.uniform(noise_range[0], noise_range[1])
    return data_component + noise_component

texts = [random.randint(0,255) for _ in range(num_traces)]
traces = [power_trace(text) for text in texts]
cw.plot(traces)

Next, let's see what happens when we group our power traces via a correct Hamming weight guess, average them, and plot the result:

In [ ]:
hamming_weight_groups = [0]*9

def trace_guesser(guess, text):
    return hamming_weight((text + guess) & 0xFF)

# iterate through each of the Hamming weights
for i in range(9):
    total = 0
    # iterate through each of the texts/power traces
    for j in range(len(texts)):
        if trace_guesser(secret_key, texts[j]) == i:
            total += 1
            hamming_weight_groups[i] += traces[j]
    if total != 0:
        hamming_weight_groups[i] /= total # divide by the total number of traces in this group to average
        
# keep list elements that don't have any entries
plot_data = []
for i in range(len(hamming_weight_groups)):
    if hamming_weight_groups[i] != 0:
        plot_data.append(hamming_weight_groups[i])
        
cw.plot(plot_data)

You should see a linear plot. Now try changing `secret_key` to another 8-bit number. You should see the linear relationship mostly disappear. This means that we can figure out the `secret_key` by trying all possible keys, then picking the one that looks the most linear!

Rather than plotting all the graphs and looking at them manually, we can instead use something called the Pearson correlation coefficient, which is a method of measuring the linearity of a dataset. It varies between -1 and 1, with -1 and 1 being perfectly linear and 0 being entirely non-linear. Note that we don't actually care whether or not the correlation is positive, we only care about he absolute value. Let's try it out on the correct key guess and an incorrect one:

In [ ]:
correct_guesses = [
    [trace_guesser(secret_key, text) for text in texts],
    list(traces)
]
correct_guesses = np.array(correct_guesses)
correct_corr = abs(np.corrcoef(correct_guesses)[1][0])

incorrect_guesses = [
    [trace_guesser((secret_key + 1) & 0xFF, text) for text in texts],
    list(traces)
]
incorrect_guesses = np.array(incorrect_guesses)
incorrect_corr = abs(np.corrcoef(incorrect_guesses)[1][0])

print(correct_corr)
print(incorrect_corr)

Next, let's try using this information to break the secret key! Make a function that takes in a key guess, the text array, and the power traces and returns the correlation coefficient. We'll use that to make a guess at the secret key:

In [ ]:
from tqdm.notebook import trange
def get_correlation(key_guess, texts, traces):
    # ###################
    # Add your code here to calculate the correlation (Code Block 3)
    # ###################

#get_correlation(2, texts, traces)

# Now we'll use get_correlation() to break the traces 

best_guess = 0
best_corr = 0
for key_guess in trange(0, 256):
    cur_corr = get_correlation(key_guess, texts, traces)
    if cur_corr > best_corr:
        best_guess = key_guess
        best_corr = cur_corr
        print("New best guess found: {:02X} (corr={})".format(best_guess, best_corr))
        
print("You guessed {:02X}, correct is {:02X}".format(best_guess, secret_key))

Hopefully you got a correct guess here. If you do get stuck, there's an answer key available. If you're way ahead, try going back to the block where we generated our power traces. Adjust the noise range, the data scaling, and the number of traces to see how they affect correlation and your ability to get a correct guess.

Note that the fact that this is a non-linear function becomes even more important with correlation, as unrelated things and guesses end up being still having a large correlation. Additionally, every guess will have the exact same correlation as its bitwise inverse (so 0xFF and 0x00 have the same correlation). 

### Applying the Attack to AES

Now that you've seen that work, you're basically ready to attack AES. While the mathematics behind the security of AES are pretty complicated, the operations within it are actually pretty simple. Even better, for this attack, we only care about the first two operations: `add_round_key()` and `sub_bytes()`. `add_round_key()` is super simple; it's just an XOR (`^` operation) between the AES state (which at the beginning is just the plaintext) and the key. As we discussed earlier, the linearity of this operation makes it difficult to attack, so let's move onto the next operation, `sub_bytes()`.

`sub_bytes()`, which is an 8-bit lookup, is basically perfect for us to attack! It's designed to be very non-linear to defend against other types of attacks, and it being an 8-bit operation means that we can break the key down into 16 8-bit chunks and attack each individually! In fact, let's only worry about one byte of the key for the rest of the lab.

This means that our `trace_guesser()` function from earlier now becomes:

```python
def trace_guesser(guess, text):
    st = add_round_key(guess, text)
    st = sub_bytes(st)
    return hamming_weight(st)
```

Now try implementing the `add_round_key()` and `sub_bytes()` functions

In [ ]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def add_round_key(key, text):
    # ###################
    # Add your code here to do add_round_key (Code Block 4)
    # ###################

def sub_bytes(state):
    # ###################
    # Add your code here to do sub_bytes (Code Block 5)
    # ###################

def trace_guesser(guess, text):
    st = add_round_key(guess, text)
    st = sub_bytes(st)
    return hamming_weight(st)

assert trace_guesser(0xA1, 0x79) == 3
assert trace_guesser(0x22, 0xB1) == 5
print("✔️ OK to continue!")

Like we said earlier, let's only worry about a single byte of the plaintext/key to attack:

In [ ]:
texts = [textin[0] for textin in textin_array]

We can make things even simpler if we only worry about a single point in each power trace. This location will be the one that you recorded in Lab 2. In my case this was `1949`, but yours might be a bit different.

In [ ]:
sbox_loc = 1949
traces = [trace[sbox_loc] for trace in trace_array]

Now that that's done, we've basically got the exact same setup as our earlier simulated setup! Recreate the `get_correlation()` function from above with the new `trace_guesser()` function, along with the guessing loop. If you've got the right code and the correct `sbox_loc`, you should get the correct key byte, `0x2b`!

In [ ]:
from tqdm.notebook import trange
# ###################
# Add your code here to guess the key (Code Block 6)
# ###################
        
print("You guessed {:02X}, correct is {:02X}".format(best_guess, key[0]))

Of course, during a real attack, we won't know where `sbox_loc` is. The fix for this is actually really simple. All you need to do is repeat the attack for every possible `sbox_loc` number, which is just `range(0, len(trace_array[0])`. Getting the rest of the key bytes is also done by repeating the attack for every `textin` in `textin_array`. This is basically what ChipWhisperer Analyzer was doing in the last lab.

We'll end this lab here, but if you're way ahead, feel free to implement the `sbox_loc` and `textin_array` extensions for a more complete attack.

---
<small>NO-FUN DISCLAIMER: This material is Copyright (C) NewAE Technology Inc., 2015-2023. ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

Tutorials derived from our open-source work must be released under the associated open-source license, and notice of the source must be *clearly displayed*. Only original copyright holders may license or authorize other distribution - while NewAE Technology Inc. holds the copyright for many tutorials, the github repository includes community contributions which we cannot license under special terms and **must** be maintained as an open-source release. Please contact us for special permissions (where possible).

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</small>